In [1]:
!pip install -U albumentations ultralytics ray hpbandster ConfigSpace "ray[tune]" optuna
!git clone https://github.com/sathishkumar67/ADIS.git
!mv /kaggle/working/ADIS/* /kaggle/working/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.6/290.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 24.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/6

In [2]:
REPO_ID = "pt-sk/Animal_Intrusion" # old dataset
FILENAME_IN_REPO = "dataset.zip"
LOCAL_DIR = "/kaggle/working/"
TRAIN_PATH = f"{LOCAL_DIR}dataset/train"
VAL_PATH = f"{LOCAL_DIR}dataset/val"
TEST_PATH = f"{LOCAL_DIR}dataset/test"
DATASET_PATH = f"{LOCAL_DIR}{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 11                                               # old class count
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', 'Duck', 'Eagle', 'Goat', 'Rodents', 'Snake', "Squirrel"] # old classes
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"
MODEL_PATH = "yolo11n.pt"

In [3]:
from huggingface_hub import hf_hub_download
from utils import unzip_file
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


dataset.zip:   0%|          | 0.00/2.60G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 2.61G/2.61G [00:13<00:00, 191MB/s] 


In [4]:
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)

In [ ]:
import optuna
from model import YOLO11Model
    

def objective(trial):
    # Define hyperparameters using Optuna suggestions
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    batch = trial.suggest_categorical('batch', [16, 32, 64])
    
    # Train YOLO model
    model = YOLO11Model(MODEL_PATH)
    results = model.train(task="detect", mode="train", data=DATA_YAML_FILE, epochs=50, lr0=lr, batch=batch)
    
    # Return validation mAP as the objective value
    return results.results_dict['metrics/mAP50-95(B)']

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=50)

In [ ]:
from ray import tune


def train_yolo(config): 
    CONFIG_DICT = {
    "task" : "detect",
    "mode" : "train",
    "data" : DATA_YAML_FILE,
    "epochs" : 10,
    "batch" : 64,
    "imgsz" : 640,
    "save" : True,
    "optimizer" : "AdamW",
    "lr0" : config["lr0"],
    "lrf" : 0.03,
    "momentum" : 0.9,
    "weight_decay" : 0.001,
    "device": 0}
    results = model.train(**CONFIG_DICT)
    tune.report(mAP=results.results_dict['metrics/mAP50-95(B)'])

analysis = tune.run(
    train_yolo,
    config={
        "lr0": tune.loguniform(1e-5, 1e-3),
    },
    num_samples=50,
    scheduler=tune.schedulers.HyperBandForBOHB(),
    search_alg=tune.search.bohb.TPE(),
)

AttributeError: module 'ray.tune.search' has no attribute 'bohb'

In [ ]:
tune.search.

In [ ]:
# Run the hyperparameter tuning
tuner = tune.Tuner(
    lambda config: model.train(**CONFIG_DICT),
    tune_config=tune.TuneConfig(
        num_samples=10,  # Number of hyperparameter combinations to try
        scheduler=bohb_hyperband,
        search_alg=bohb_search,
    ),
)
results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

In [ ]:
# import torch
# import gc

# gc.collect()  # Run garbage collector
# torch.cuda.empty_cache()  # Free unused memory
# torch.cuda.memory_allocated(), torch.cuda.memory_reserved()
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()